# <center>OpenAI在线大模型调用及微调方法

## <center>Ch.5 借助LtM提示工程攻克SCAN指令翻译难题

### 1.项目背景及SCAN数据集介绍

&emsp;&emsp;根据Ch.3提示工程入门部分内容，我们了解，到对于大语言模型来说，组合泛化能力是最为重要的涌现能力之一，也是模型能够理解复杂语义、进行准确推理的底层能力，也是影响模型编程、翻译以及其他各类NLP任务执行能力最为重要的底层能力。因此，某个具体提示工程技术是否有效，最有效的方式就是观察该方法是否能有效提升模型组合泛化能力。在上一小节中，我们首先介绍了一系列基础提示工程方法，包括Few-shot、CoT方法和LtM方法等，并在一些基础推理示例上验证了这些方法对于提升模型组合泛化能力的有效性（推理能力属于组合泛化能力）。当然，就上一小节的教学示例来说，都属于非常基础的推理问题，实际使用的提示工程流程也并不复杂。那么接下来，我们继续尝试把这些提示工程方法应用到一个更加复杂的工业级实战场景中，一方面进一步介绍提示工程的复杂实战技巧，另一方面也通过一个工业级实战案例，为大家展示提示工程+大预言模型的实战威力。

&emsp;&emsp;本小节介绍的实战项目为指令翻译项目，采用的数据集为纽约大学和Facebook的科学家们于2018年提出的SCAN数据集，该数据集包含2万条基于隐藏语义关系的指令及对应的翻译动作序列，是目前公认的用于测试深度学习算法组合泛化能力的基准数据集。尽管数据集是人工创建的数据集，但该数据集建模难度极大，哪怕是目前最前沿的深度学习算法都无法进行非常精准的预测，在大模型诞生之前，人们甚至需要单独围绕这个数据集提出全新的深度学习架构，才能完成较高进度的预测。而这也使得该数据集也是目前公认的最为有效的测试模型组合泛化能力的数据集。接下来我们尝试使用上一小节介绍的提示工程方法，在text-davinci-003模型上完成对该数据集的预测。这是课程中第一个基于大模型的工业级实战项目，也是我们首次基于大模型来完成一个完整的建模流程。

&emsp;&emsp;需要注意的是，围绕该问题的大语言模型建模预测最早见于论文[《LEAST-TO-MOST PROMPTING ENABLES COMPLEX REASONING IN LARGE LANGUAGE MODELS》](https://arxiv.org/pdf/2205.10625.pdf)，改论文也是上一小节介绍的LtM方法提出论文，在该论文中，采用了一种较为复杂的LtM提示方法，最终完成了准确率高达99.7%的预测。本节内容我们将先尝试复现改论文的方法，然后尝试对其进行改进。

- SCAN数据集形式

&emsp;&emsp;SCAN（Simplified Compositional Actions and Navigation）数据集由纽约大学和Facebook的科学家们于2018年提出，数据集本身是一系列的指令和对应的行为序列组成的数据集，指令在数据集中以IN的形式进行申明，是一串描述动作序列的英文句子，例如"walk twice and jump twice"；而行为序列则是以OUT的形式进行申明，是指令所指代的一系列精准步骤，例如“"WALK", "WALK", "JUMP", "JUMP"”，数据集基本结构如下：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306291118609.png" alt="a358c4956cbd8964e7d39b57399ab1d" style="zoom:50%;" />

不难看出，这是一个纯文本数据集。SCAN数据集总共包含2万条样本（2万个IN-OUT对）。对比传统的结构化数据集，IN相当于是特征，OUT相当于是标签。

- SCAN数据集测试模型组合泛化能力

&emsp;&emsp;从自然语言的角度，有些指令和行为之间的关系较好理解，例如“walk twice and jump twice”当然就是指代“"WALK", "WALK", "JUMP", "JUMP"”，但有些命令和行为之间的关系却不是很好理解，例如上图中的最后一个命令：“walk opposite left twice and run”，为何是指代“"TURN_LEFT","TURN_LEFT","WALK","TURN_LEFT","TURN_LEFT","WALK","RUN"”，其实并不能通过自然语言的语法规则进行理解。实际上，这个数据集并不是完全根据自然语言逻辑设置的指令和行为之间的关系，而是根据一套隐藏的语法规则创建的指令和行为数据集，换而言之指令和动作在现实世界中并没有特殊的含义，但它们模拟了一种常见的问题：理解和执行一系列指令。这种问题在很多实际应用中都会遇到，例如：编程（理解并执行一系列代码指令）、机器人导航（理解并执行一系列的导航指令），甚至是日常生活中的任务（如烹饪、装配家具等，需要理解并执行一系列步骤）。

&emsp;&emsp;据此不难理解，SCAN这个人工数据集创建的目的，就是为了测试模型能否找到这个隐藏的语法规则，并根据IN准确的预测OUT，预测准确率越高，说明模型对这个隐藏语法规则学习的越准确，模型性能越强。而围绕着某个规则系统进行学习和预测的能力，在机器学习专业领域被称为模型的组合泛化能力。从专业的角度来说，SCAN数据集就是一个用于评估模型在理解和执行指令上的组合泛化能力的基准数据集。

- SCAN数据集建模难度

&emsp;&emsp;机器学习（深度学习）模型的组合泛化能力属于目前非常前沿的技术领域，在很多应用领域如机器翻译、对话系统和机器人导航等，都需要具备强大的组合泛化能力模型作为算法引擎。而SCAN数据集则是目前建模难度非常大的数据集，主要原因由以下两点：      
&emsp;&emsp;其一，从数据集本身的角度而言，由于SCAN数据集数据集的主要特点是其指令和相应的行动序列的生成，并不依赖于真实世界的语义，而是完全基于某种预定义的、结构化的语法规则。这使得模型不能仅依赖于表面层次的统计规律或者局部的模式匹配来进行预测，而必须理解和掌握数据的底层结构和规则；      
&emsp;&emsp;其二，从模型本身的角度来说，深度学习模型、特别是序列到序列的模型，通常依赖于大量的数据和计算资源来学习和泛化复杂的模式，而SCAN数据集则需要模型具备强大的组合泛化能力，即只通过少量数据集的训练，就需要理解没有直接见过的组合指令并执行正确的行动序列。例如，虽然模型可能在训练中见过"jump"和"twice"这两个单词，但是如果没有直接见过"jump twice"这个组合，那么模型可能就无法正确地理解和执行这个指令。       
&emsp;&emsp;因此截至目前，传统的深度学习方法、哪怕是最先进的深度学习模型也难以在该数据集上达到高水平的性能。因此，业内甚至围绕类似SCAN的预测任务提出了全新的神经网络模型，例如记忆增强的神经模型LANE等。也由此可见SCAN数据集建模难度之大。 

&emsp;&emsp;但就是这样的一个业内难题，却能够被大语言模型“轻易”的解决。根据《LEAST-TO-MOST PROMPTING ENABLES COMPLEX REASONING IN LARGE LANGUAGE MODELS》论文中的描述，根据基于合理的LtM提示学习，"code-davinci-002"模型只需要输入14条训练样本，就能够对SCAN数据集做到几乎100%的准确预测（准确率为99.7%）。由此足见大模型的威力，而传统NLP领域对大模型的技术恐慌并非空穴来风。

&emsp;&emsp;这里需要注意，大语言模型在全新的语法规则上的组合泛化能力本质上也是一种“涌现能力”。大语言模型的预训练预料都来自于自然语言，模型并未接收过SCAN这种特定指令数据集的训练，因此模型对特定语法规则的理解可以看成是对自然语言理解能力的迁移，因此本质上也是一种涌现能力，只不过这种涌现能力需要合理的提示学习来进行激发。

&emsp;&emsp;接下来，我们尝试复现论文中的LtM提示工程以及实际预测过程。需要注意的是，原论文中的很多环节并未给出详细的解释，课程中我们将结合当前实践经验进行更加具体的解释，同时也给出用于解决更多类似任务的LtM提示工程建议。

- SCAN数据集下载

&emsp;&emsp;SCAN数据集目前托管在Hugging Face上，我们可以借助Hugging Face的datasets库进行在线下载和导入。首次使用datasets库可以按照如下方法进行安装：

In [ ]:
!pip install datasets

安装完成后，即可按照如下方式导入SCAN数据集：

In [3]:
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
from datasets import load_dataset

In [5]:
# 加载数据集
dataset = load_dataset("scan", "simple")

# 打印数据集信息
print(dataset)

Dataset scan downloaded and prepared to C:/Users/Administrator/.cache/huggingface/datasets/scan/simple/1.0.0/68e9bb6106d45ba696314f2dea52a1e893f99cbf422feebbeda2f1d11878e020. Subsequent calls will reuse this data.


100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 570.19it/s]

DatasetDict({
    train: Dataset({
        features: ['commands', 'actions'],
        num_rows: 16728
    })
    test: Dataset({
        features: ['commands', 'actions'],
        num_rows: 4182
    })
})


其中dataset是完整数据集，且已经被划分为训练集和测试集两个数据集，其中训练集总共包含16728条数据，而测试集总共包含4182条数据。该数据集类型类似于字典，我们可以使用字典索引的方式单独查看数据集中的训练集和测试集：

In [6]:
scan_train = dataset['train']
scan_train

Dataset({
    features: ['commands', 'actions'],
    num_rows: 16728
})

In [7]:
scan_test = dataset['test']
scan_test

Dataset({
    features: ['commands', 'actions'],
    num_rows: 4182
})

而此时的scan_train和scan_test对象类型类似于List，我们可以直接通过索引得到其中的每一行数据：

In [8]:
scan_train[0]

{'commands': 'jump opposite right twice and turn opposite right thrice',
 'actions': 'I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT'}

并且发现，其中的每一行数据的保存形式都是字典，该字典包含两个Key-Value对，第一个键值对存储了命令信息，第二个键值对则存储了命令对应的行为序列。接下来我们的任务就是借助大语言模型和LtM提示过程，准确的围绕每个Commands进行actions的预测。不难看出，这套数据集中的指令和行为之间并不完全遵照自然语言规则，对于模型来说若要进行准确预测，则需要尝试学习其背后的隐藏语法规则。

&emsp;&emsp;这里需要注意，hugging face中的Dataset数据集和pandas数据集是能够顺畅进行转化的，例如我们可以按照如下方法scan_train转化为pandas对象类型，此时列名称就是字典中的key，而对应的value值就是现在每一列具体的取值：

In [146]:
scan_train.to_pandas()

,commands,actions
0,jump opposite right twice and turn opposite ri...,I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT ...
1,run opposite left after walk right,I_TURN_RIGHT I_WALK I_TURN_LEFT I_TURN_LEFT I_RUN
2,walk after run around right twice,I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_R...
3,look around right thrice and turn left,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN...
4,walk opposite left twice and walk opposite rig...,I_TURN_LEFT I_TURN_LEFT I_WALK I_TURN_LEFT I_T...
...,...,...
16723,look right after jump opposite right twice,I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT ...
16724,run opposite right thrice after run right twice,I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_R...
16725,run right thrice and turn around right twice,I_TURN_RIGHT I_RUN I_TURN_RIGHT I_RUN I_TURN_R...
16726,run opposite right thrice and walk twice,I_TURN_RIGHT I_TURN_RIGHT I_RUN I_TURN_RIGHT I...


### 2.针对SCAN数据预测的LtM提示流程

#### 2.1 思维链&Zero-shot-LtM提示工程局限性

&emsp;&emsp;面对这个异常复杂的“推理”问题，我们首先可以尝试上一小节介绍的系列提示工程方法。这里我们在现在数据集中挑选几条相对简单的数据：

|Command|Action Sequence|      
|:--:|:--:| 
|“look thrice after jump”|JUMP LOOK LOOK LOOK| 
|“run left and walk”|TURN LEFT RUN WALK| 
|“look opposite right”|TURN RIGHT TURN RIGHT LOOK| 

In [32]:
Command1 = 'look thrice after jump' 
Action1 = 'JUMP LOOK LOOK LOOK'

In [33]:
Command2 = 'run left and walk' 
Action2 = 'TURN LEFT RUN WALK'

In [34]:
Command3 = 'look opposite right' 
Action3 = 'TURN RIGHT TURN RIGHT LOOK'

然后首先尝试Zero-shot提示方法：

In [35]:
response_Zero_shot = openai.Completion.create(
                          model="text-davinci-003",
                          prompt=Command1,
                          max_tokens=1000,
                          )

response_Zero_shot["choices"][0]["text"].strip()

',\n\n1. Make sure that your landing zone is clear of any objects or people.\n2. During the jump, check for any potential dangerous areas that you could land in such as water, puddles, or furniture.\n3. Look for any obstacles that may block your path as you land, such as furniture, trees, or walls, and make sure to adjust your angle as necessary to prevent any mishaps.'

能够发现，模型完全无法给出答案。仔细思考不难发现，由于当前翻译问题根本不是自然语言语境下的问题，模型甚至不知道应该如何进行“补全”。哪怕是我们在提示中输入让模型进行翻译，模型也不知道应该如何进行翻译：

In [37]:
"'%s' should be translated as" % Command1

"'look thrice after jump' should be translated as"

In [39]:
response_Zero_shot = openai.Completion.create(
                          model="text-davinci-003",
                          prompt= "%s should be translated as" % Command1,
                          max_tokens=1000,
                          )

response_Zero_shot["choices"][0]["text"].strip()

'mira tres veces después de saltar'

模型将其翻译为了西班牙语。

&emsp;&emsp;很明显，针对非自然语境下的特定问题，我们至少需要通过Few-shot的方式让模型先学习、后预测。接下来我们再尝试以第一、二条数据作为few-shot，围绕第三条数据进行预测，结果如下：

In [43]:
few_shot_prompt = 'Q: "%s", A: "%s", Q: "%s", A: "%s", Q: "%s", A: ' % (Command1, Action1, Command2, Action2, Command3)
few_shot_prompt

'Q: "look thrice after jump", A: "JUMP LOOK LOOK LOOK", Q: "run left and walk", A: "TURN LEFT RUN WALK", Q: "look opposite right", A: '

In [44]:
response_Few_shot = openai.Completion.create(
                          model="text-davinci-003",
                          prompt=few_shot_prompt,
                          max_tokens=1000,
                          )

response_Few_shot["choices"][0]["text"].strip()

'"LOOK LEFT"'

In [45]:
Action3

'TURN RIGHT TURN RIGHT LOOK'

能够发现，简单的以问题+答案组成的few-shot，在当前SCAN数据集上仍然无法进行较为准确的预测。事实上，根据原论文的描述，简单的few-shot提示方法在SCAN数据集上的准确率不到17%。而关于Few-shot本身的能力局限，我们在上一小节解决推理问题的过程中也是得到了相同的验证。

&emsp;&emsp;而根据原论文的描述，（尽管没有给出实际的提示示例，但）Few-shot-CoT方法效果也达不到17%，为解决这一预测问题，最佳方法就是进行LtM提示工程，同时是一种基于Few-shot的LtM提示工程方法，我们也可以将其称为Few-shot-LtM，而上一小节介绍的LtM方法则可以视为Zer-shot-LtM方法。这里我们可以首先验证Zero-shot-LtM在当前数据集上的无效性。这里我们简单回顾上一小节LtM提示方法的基本流程：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306292044573.png" alt="4173ae0afb29936452104fc0281a9f4" style="zoom:33%;" />

接下来，我们根据"to solve...，we need to first solve"提示模板来进行问题拆解：

In [50]:
Zero_shot_LtM_prompt = "In order to translate '%s', we need to first solve" % Command1
Zero_shot_LtM_prompt

"In order to translate 'look thrice after jump', we need to first solve"

In [51]:
response_Zero_shot_LtM = openai.Completion.create(
                          model="text-davinci-003",
                          prompt=Zero_shot_LtM_prompt,
                          max_tokens=1000,
                          )

response_Zero_shot_LtM["choices"][0]["text"].strip()

"for the idiom. The correct translation would be to 'Look twice before you leap'."

和上一小节看到的结果类似，模型直接围绕当前问题进行子问题拆解，将其“拆解为（理解为）”一个成语翻译问题，并最终输出翻译结果：该指令代表三思而后行这一成语。

&emsp;&emsp;很明显，在Zero-shot-LtM的情况下，模型的核心问题仍然还是无法精准理解问题，更别谈进行有效的潜在语义关系学习了。当然，究其原因还是因为对于很多完全脱离自然语言语义规则的问题，简单提示模板是很难让模型完成有意义的问题拆解的，例如对于SCAN数据集中的指令翻译问题，由于本身指令翻译的规则就不是自然语言规则，模型也从未学习过相关规则，此时在Zero-shot的情况下是很难让模型进行问题拆解的，或者说此时模型拆解的问题也几乎不会有助于最终的指令翻译任务，拆解的子问题是毫无意义的。

#### 2.2 Few-shot-LtM提示工程基本流程

&emsp;&emsp;接下来，论文中提出了一种Few-shot-LtM提示工程流程来解决该问题，这种Few-shot-LtM提示流程在原有流程上进行了以下两方面改进：

- 首先通过引入Few-shot来引导模型进行有价值的中间问题拆解并进行回答。对于完全脱离自然语言语境下的推理问题，这其实是非常重要的一个环节，我们只有给了模型非常准确率的引导，才能够帮模型打开学习潜在语义的窗口。但是，这实际上也是非常复杂且具有创造力的一步，不难理解，通过自然语言描述的拆分问题的方法是无穷无尽的，当然也没有那种方法是标准答案。因此，这方面的Few-shot的设计就会需要建模人员大量的工作经验+大量的尝试，来寻找到最佳的Few-shot拆分示例，由此也能看出提示工程也是人工创造力灵感和前沿技术的完美结合；

- 而同时在Few-shot的示例中，尝试引导模型进行多步分解进而提升模型进行翻译的准确率。即Decompose Question into Subquestions环节，可以尝试多步拆分多个子问题回答，即我们可以尝试围绕某个拆解之后的子问题进一步拆分子问题，进而提升子问题的回答准确率，例如假设将一个问题拆分成三个子问题，则LtM提示回答的基本过程如下
<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/202306292119088.png" alt="0660614d66dc5b7e0a82398882664da" style="zoom:40%;" />

注意，这里的Few-shot实际上是两个部分，我们既需要利用Few-shot告诉模型应该如何拆解问题，同时也需要通过Few-shot来告诉模型应该如何进行问题的回答。接下来，我们先来看原论文是如何在SCAN数据集上进行Few-shot-LtM提示工程的，然后再探讨其背后的原因。

#### 2.3 Few-shot-LtM提示工程在SCAN数据集上的提示流程

&emsp;&emsp;针对SCAN数据集，论文中将Few-shot-LtM提示的两个阶段进行重新命名，此前的Decompose Questions into Subquestions被重新命名为Command decomposition，即命令分解，其实也就是问题拆解，而第二个阶段Sequentially Solve Subquestion则重新命名为Command mapping，即指令翻译，当然其实本质也就是依次翻译拆解的指令以及原始指令。而这两个阶段的新名称，也非常清晰的指明了对于SCAN数据集来说，LtM提示工程的具体应用形式就是——先拆分命令、再逐个进行翻译。

##### 2.3.1 Stage 1.Command decomposition：指令拆解

&emsp;&emsp;首先第一步，采用Few-shot的方法围绕命令进行拆解。这里需要注意的是，Few-shot中的提示示例是人工手动编写，每个示例都是借助"__ can be solved by:__"提示模板进行提问，进而引导模型对Commands进行问题拆解，即为了解读这个长的指令，我们可以首先解读哪些与之相关的短指令。尽管实际上是问题拆解，但从表现形式上来看，最终是将一个长指令拆解为一组短指令，因此这个过程也被称为指令拆解的过程（Command decomposition）。这里论文挑选了几个指令并进行拆解，拆解过程如下：

In [184]:
CD_Few_shot = 'Q: “look opposite right thrice after walk” \
               A: “look opposite right thrice” can be solved by: “look opposite right”, “look opposite right \
               thrice”. “walk” can be solved by “walk”. So, “look opposite right thrice after walk” can be \
               solved by: “walk”, “look opposite right”, “look opposite right thrice”. \
               Q: “look around right thrice and walk” \
               A: “look around right thrice” can be solved by: “look right”, “look around right”, “look around \
               right thrice”. “walk” can be solved by “walk”. So, “look around right thrice and walk” can be \
               solved by: “look right”, “look around right”, “look around right thrice”, “walk”. \
               '

这组提示示例相当于告诉模型应该如何进行指令拆解、即告诉模型指令拆解的基本规范，例如“look opposite right thrice after walk”可以被拆解为“walk”, “look opposite right”, “look opposite right thrice”这三个短的指令，而“look around right thrice and walk”，则可以被拆解为“look right”, “look around right”, “look around right thrice”, “walk”。

&emsp;&emsp;具体为了这么拆解指令，稍后我们会有深度探讨。这里我们需要重点关注的是这些提示示例并不是普通的示例，一般情况下Few-shot是Q为问题而A为答案，例如实际上，Command：“look opposite right thrice after walk”对应的Action为“I_WALK I_TURN_RIGHT I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_TURN_RIGHT I_LOOK”，而Commend“look around right thrice and walk”则对应的Action为“I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_WALK”。即如果按照普通的Few-shot，此时提示示例应该为：

In [72]:
'Q: “look opposite right thrice after walk” \
A: “I_WALK I_TURN_RIGHT I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_TURN_RIGHT I_LOOK”. \
Q: “look around right thrice and walk” \
A: “I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK \
I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT \
I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT \
I_LOOK I_TURN_RIGHT I_LOOK I_WALK”. \
'

'Q: “look opposite right thrice after walk” A: “I_WALK I_TURN_RIGHT I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_TURN_RIGHT I_LOOK”. Q: “look around right thrice and walk” A: “I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_WALK”. '

但LtM提示流程的不同之处就在于设置了多个阶段进行分段提示，而第一个阶段的提示结果只是为了创建中间结果——即分解子问题（命令拆解），因此Few-shot-LtM第一个阶段的提示示例也就是展示如何进行指令拆解。但需要注意的是，由于中间结果并没有统一的标准答案，因此根据这些示例拆解的中间结果是否有效，其实是有待最后的预测结果输出来进行验证的。并且，无论中间结果是否有效，由于中间结果并没有标准答案，因此如何拆解基本可以看成是人工尝试+经验的总结；而更为重要的是，无论中间结果投入到最终环节进行验证后验证结果如何，这种中间环节的Few-shot都是极为灵活的，对于大语言模型来说，不同的提示模板、语言风格、甚至是不同提示示例，都会对模型造成不同的引导，这种引导是极为灵活且具有创造力的。为了获得一个更好的最终结果，中间环节的Few-shot或许要经历千百次的尝试，而在这个过程中，是及其考验大模型工程师（提示工程师）本身的创造力和经验的。当然，无法通过一个规定死的流程来解决问题，而需要大量的人工经验和创造力，这既是对人的考验，同时也是大模型工程师岗位本身的价值所在，这也是提示工程技术最为核心的价值。

> 当然，这种灵活设置中间结果提示示例的过程，也可以看成是在大模型中“掘金”。

> 不过本例中的提示示例设置方法也并非完全无迹可寻，稍后我们会深度探讨如此设置中间环节提示背后的原因。

> 很多时候Few-shot就可以看成是训练集，大语言模型的建模过程和机器学习类似，都是在训练集上进行训练，然后在新的数据集上验证效果。

&emsp;&emsp;很明显，在CD_Few_shot的方法提示下，接下来对于很多指令，模型就知道了到底应该如何拆解，例如我们尝试在上述Few-shot的情况下，引导模型拆解SCAN数据集中的第二条指令，拆解过程如下，这里我们为了方便表示，直接把第二条数据集的commands是做模型输入X，而把对应的actions是为模型输出y，则第二条数据集可以通过如下形式进行表示：

In [185]:
scan_train[1]

{'commands': 'run opposite left after walk right',
 'actions': 'I_TURN_RIGHT I_WALK I_TURN_LEFT I_TURN_LEFT I_RUN'}

In [186]:
scan_train[1]['commands']

'run opposite left after walk right'

In [187]:
X = scan_train[1]['commands']
X

'run opposite left after walk right'

In [188]:
y = scan_train[1]['actions']
y

'I_TURN_RIGHT I_WALK I_TURN_LEFT I_TURN_LEFT I_RUN'

接下来围绕该条数据集进行拆解：

In [189]:
'Q：“%s” A:' % X

'Q：“run opposite left after walk right” A:'

In [190]:
prompt_CD = CD_Few_shot + 'Q：“%s” A:' % X
prompt_CD

'Q: “look opposite right thrice after walk”                A: “look opposite right thrice” can be solved by: “look opposite right”, “look opposite right                thrice”. “walk” can be solved by “walk”. So, “look opposite right thrice after walk” can be                solved by: “walk”, “look opposite right”, “look opposite right thrice”.                Q: “look around right thrice and walk”                A: “look around right thrice” can be solved by: “look right”, “look around right”, “look around                right thrice”. “walk” can be solved by “walk”. So, “look around right thrice and walk” can be                solved by: “look right”, “look around right”, “look around right thrice”, “walk”.                Q：“run opposite left after walk right” A:'

In [191]:
response_CD = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt_CD,
              temperature=0.5,
              max_tokens=1000
              )

In [192]:
response_CD["choices"][0]["text"].strip()

'“run opposite left” can be solved by: “run left”, “run opposite left”. “walk right” can be solved by: “walk right”. So, “run opposite left after walk right” can be solved by: “walk right”, “run left”, “run opposite left”.'

能够看出，原始指令'run opposite left after walk right'就被拆分成了“walk right”, “run left”, “run opposite left”三个短指令。当然，从中也能发现，Few-shot的提示过程其实也可以看成是让模型举一反三的过程，尽管训练数据中没有'run opposite left after walk right'这个指令拆解的方法，但通过Few-shot，模型学会了其底层拆解命令的逻辑，因此也顺利完成了对这个新指令的拆解。

&emsp;&emsp;此外，需要注意的是，这里模型将X拆分成三个子命令，就相当于是将一个原始问题拆分成了三个问题，并且需要注意的是，这三个问题是有先后顺序的，相当于是“walk right”是最底层的子问题、“run left”是上一层的子问题，“run opposite left”则是第一层子问题。这里之所以要注意子问题的问题顺序，是因为后续在解决这些子问题的时候，是按照由下往上的顺序依次进行解决，并且每一个子问题的解决都需要将下一层子问题的问题+答案作为Few-shot（最底层的子问题除外），因此我们需要注意拆分问题的顺序。

> 当然，拆解命令是否真的能帮模型完成对完整指令的预测，还需借助第二阶段指令翻译最终的输出结果进行验证。

##### 2.3.2 Stage 2.Command mapping：指令翻译

&emsp;&emsp;在第一个阶段，当我们成功的围绕原问题（原命令）拆分除了多个子问题（短命令）之后，接下来就需要依次解答（翻译）这一系列的子问题。这个阶段总共有两部分工作，其一是通过Few-shot教会模型短命令的翻译方法，其二则是在翻译原始指令时，需要先翻译短指令，然后将短指令的问题和答案都作为prompt的一部分，带入到原指令的翻译过程中。其中，第二部分工作也就是LtM提示方法的根本流程——将子问题的问题+答案作为原问题提示内容的一部分，从而让模型更好的解决原问题。

&emsp;&emsp;我们先来看短指令的Few-shot。同样，研究人员挑选了部分短指令，并制定了短指令翻译的一般方法，部分短指令翻译的示例如下：

In [193]:
CM_Few_shot = 'Q: “jump left” \
               A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn \
               left” outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn \
               left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” \
               is “TURN LEFT” + “JUMP”. \
               Q: “run and look twice” \
               A: The output of “run and look twice” concatenates: the output of “run”, the output of “look \
               twice”. “run” outputs “RUN”. “look twice” outputs “LOOK” * 2. So concatenating the output of \ \
               “run” and the output of “look twice” leads to “RUN” + “LOOK” * 2. So the output of “run and \
               look twice” is “RUN” + “LOOK” * 2. \
               Q: “walk opposite left” \
               A: The output of “walk opposite left” concatenates: the output of “turn opposite left”, the output of \
               “walk”. “turn opposite left” outputs “TURN LEFT” * 2. “walk” outputs “WALK”. So concatenating the \
               output of “turn opposite left” and the output of “walk” leads to “TURN LEFT” * 2 + “WALK”. So the \
               output of “walk opposite left” is “TURN LEFT” * 2 + “WALK” '

从结果上来看，这些提示就是通过两阶段的逻辑引导，将一些相对较短的指令翻译成了最终行为序列。

&emsp;&emsp;那么接下来，我们就借助这些翻译的示例，尝试对测试数据X完成Command mapping过程，这里再次强调，X的Command mapping流程是先借助CM_Few_shot对最底层的子命令进行翻译，然后再把这一层子命令的问题+答案添加进CM_Few_shot中，再对上一层子问题进行回答，并按照这个流程最终回答原始问题。具体执行过程如下：

- 翻译第一个子命令walk right：

&emsp;&emsp;首先是翻译第一个子命令“walk right”，借助CM_Few_shot对第一个子命令翻译过程如下：

In [194]:
prompt_CM_1 = CM_Few_shot + 'Q:“walk right” A：'

In [195]:
response_CM_1 = openai.Completion.create(
                model="text-davinci-003",
                prompt=prompt_CM_1,
                temperature=0.5,
                max_tokens=1000
                )

In [196]:
response_CM_1["choices"][0]["text"].strip()

'The output of “walk right” concatenates: the output of “turn right”, the output of “walk”. “turn right” outputs “TURN RIGHT”. “walk” outputs “WALK”. So concatenating the output of “turn right” and the output of “walk” leads to “TURN RIGHT” + “WALK”. So the output of “walk right” is “TURN RIGHT” + “WALK”.'

上述结果既是第一个子命令的翻译得到结果。

- 翻译第二个子命令run left：

&emsp;&emsp;而在翻译第二个子命令时，则需要将第一个子命令时的问答结果作为Few-shot的一个示例，即我们需要先把第一个子命令的问答结果拼接到prompt_CM_1中，然后在提出第二个子命令翻译问题：

In [197]:
prompt_CM_2 = prompt_CM_1 + response_CM_1["choices"][0]["text"].strip() + 'Q:“run left” A：'

In [198]:
response_CM_2 = openai.Completion.create(
                model="text-davinci-003",
                prompt=prompt_CM_2,
                temperature=0.5,
                max_tokens=1000
                )

In [199]:
response_CM_2["choices"][0]["text"].strip()

'The output of “run left” concatenates: the output of “turn left”, the output of “run”. “turn left” outputs “TURN LEFT”. “run” outputs “RUN”. So concatenating the output of “turn left” and the output of “run” leads to “TURN LEFT” + “RUN”. So the output of “run left” is “TURN LEFT” + “RUN”.'

由此，便得到了第二个子命令的答案。

- 翻译第三个子命令run opposite left：

&emsp;&emsp;而第三个子命令的翻译，则需要同时将此前两个子命令的问答都加入Few-shot中，然后再进行提问：

In [200]:
prompt_CM_3 = prompt_CM_2 + response_CM_2["choices"][0]["text"].strip() + 'Q:“run opposite left” A：'

In [201]:
response_CM_3 = openai.Completion.create(
                model="text-davinci-003",
                prompt=prompt_CM_3,
                temperature=0.5,
                max_tokens=1000
                )

In [202]:
response_CM_3["choices"][0]["text"].strip()

'The output of “run opposite left” concatenates: the output of “turn opposite left”, the output of “run”. “turn opposite left” outputs “TURN LEFT” * 2. “run” outputs “RUN”. So concatenating the output of “turn opposite left” and the output of “run” leads to “TURN LEFT” * 2 + “RUN”. So the output of “run opposite left” is “TURN LEFT” * 2 + “RUN”.'

由此便完成了第三个子命令的问答过程。

- 原始问题run opposite left after walk right问答过程

&emsp;&emsp;在获得了每个子命令的问答结果之后，接下来，将每个子命令的问答结果都拼接到Few-shot中，并对模型提问原始问题，流程如下：

In [203]:
prompt_CM = prompt_CM_3 + response_CM_3["choices"][0]["text"].strip() + 'Q：“%s” A:' % X
prompt_CM

'Q: “jump left”                A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn                left” outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn                left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left”                is “TURN LEFT” + “JUMP”.                Q: “run and look twice”                A: The output of “run and look twice” concatenates: the output of “run”, the output of “look                twice”. “run” outputs “RUN”. “look twice” outputs “LOOK” * 2. So concatenating the output of \\                “run” and the output of “look twice” leads to “RUN” + “LOOK” * 2. So the output of “run and                look twice” is “RUN” + “LOOK” * 2.                Q: “walk opposite left”                A: The output of “walk opposite left” concatenates: the output of “turn opposite left”, the output of                “walk”. “turn opposite left” outputs 

In [204]:
response_CM = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt_CM,
              temperature=0.5,
              max_tokens=1000
              )

In [205]:
response_CM["choices"][0]["text"].strip()

'The output of “run opposite left after walk right” concatenates: the output of “walk right”, the output of “run opposite left”. “walk right” outputs “TURN RIGHT” + “WALK”. “run opposite left” outputs “TURN LEFT” * 2 + “RUN”. So concatenating the output of “walk right” and the output of “run opposite left” leads to “TURN RIGHT” + “WALK” + “TURN LEFT” * 2 + “RUN”. So the output of “run opposite left after walk right” is “TURN RIGHT” + “WALK” + “TURN LEFT” * 2 + “RUN”.'

最终，我们能够获得原始问题的答案为“TURN RIGHT” + “WALK” + “TURN LEFT” * 2 + “RUN”，仔细观察不难发现，该回答和该条指令的真实标签一致：

In [206]:
y

'I_TURN_RIGHT I_WALK I_TURN_LEFT I_TURN_LEFT I_RUN'

当让，我们也可以借助如下函数将response_CM["choices"][0]["text"].strip()对象转化为y格式：

In [269]:
import re

def transform_expression(s):
    # Regular expression pattern
    pattern = r'is “.*'

    # Find the match
    match = re.search(pattern, s) 
    
    s = match.group()[3: -1].replace('“', '"').replace('”', '"')
    # Step 1: Handle multiplications
    pattern = r'"([^"]+)" \* (\d+)'
    matches = re.findall(pattern, s)
    for match in matches:
        replacement = ' '.join([f'"{match[0]}"'] * int(match[1]))
        s = s.replace(f'"{match[0]}" * {match[1]}', replacement)

    # Step 2: Replace spaces within quotes with underscores
    pattern = r'"([^"]+)"'
    matches = re.findall(pattern, s)
    for match in matches:
        replacement = match.replace(' ', '_')
        s = s.replace(f'"{match}"', f'"{replacement}"')

    # Step 3: Add 'I_' prefix within quotes
    pattern = r'"([^"]+)"'
    matches = re.findall(pattern, s)
    for match in matches:
        replacement = 'I_' + match
        s = s.replace(f'"{match}"', f'"{replacement}"')

    # Step 4: Remove quotes
    s = s.replace('"', '')
    s = s.replace(' +', '')

    return s

In [208]:
transform_expression(response_CM["choices"][0]["text"].strip())

'I_TURN_RIGHT I_WALK I_TURN_LEFT I_TURN_LEFT I_RUN'

In [209]:
y

'I_TURN_RIGHT I_WALK I_TURN_LEFT I_TURN_LEFT I_RUN'

由此，我们就完成了一次完整的基于复杂语义问题的LtM提示工程流程，并顺利获得准确答案。不难看出，Few-shot-LtM提示流程非常复杂，以下是X到y翻译过程总结：

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1688807852615.png" alt="1688807852615" style="zoom:33%;" />

### 3.SCAN数据集完整预测流程

&emsp;&emsp;在了解了Few-shot-LtM的基本流程之后，我们尝试用代码实现一个更加自动化的Few-shot-LtM过程，并完成在完整的SCAN数据集上建模的流程。

- 提取数据流

In [85]:
scan_train

Dataset({
    features: ['commands', 'actions'],
    num_rows: 16728
})

In [86]:
scan_test

Dataset({
    features: ['commands', 'actions'],
    num_rows: 4182
})

In [87]:
scan_train[0]

{'commands': 'jump opposite right twice and turn opposite right thrice',
 'actions': 'I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT'}

In [210]:
for data in scan_test:
    print(data['commands'])
    print(data['actions'])
    break

turn opposite right thrice and turn opposite left
I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT


In [211]:
data

{'commands': 'turn opposite right thrice and turn opposite left',
 'actions': 'I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT'}

- 提示示例补充

&emsp;&emsp;此外需要注意的是，根据原论文的描述，为了更好的完成完整数据集的预测，第一阶段命令拆解需要设置8组问答示例，而第二阶段命令映射则需要设置14组问答示例，因此这里我们完整补充第一阶段的提示示例：

In [216]:
CD_Few_shot = 'Q: “look right after look twice” \
               A: “look right after look twice” can be solved by: “look right”, “look twice”. \
               Q: “jump opposite right thrice and walk” \
               A: “jump opposite right thrice” can be solved by: “jump opposite right”, “jump opposite right thrice”. \
               “walk” can be solved by: “walk”. So, “jump opposite right thrice and walk” can be solved by: “jump \
               opposite right”, “jump opposite right thrice”, “walk”. \
               Q: “run left twice and run right” \
               A: “run left twice” can be solved by: “run left”, “run left twice”. “run right” can be solved by “run right”. \
               So, “run left twice and run right” can.be solved by: “run left”, “run left twice”, “run right”. \
               Q: “run opposite right” \
               A: “run opposite right” can be solved by “run opposite right”. \
               Q: “look opposite right thrice after walk” \
               A: “look opposite right thrice” can be solved by: “look opposite right”, “look opposite right thrice”. \
               “walk” can be solved by “walk”. So, “look opposite right thrice after walk” can be solved by: “look \
               opposite right”, “look opposite right thrice”, “walk”. \
               Q: “jump around right” \
               A: “jump around right” can be solved by: “jump right”, “jump around right”. So, “jump around right” \
               can be solved by: “jump right”, “jump around right”. \
               Q: “look around right thrice and walk” \
               A: “look around right thrice” can be solved by: “look right”, “look around right”, “look around right \
               thrice”. “walk” can be solved by “walk”. So, “look around right thrice and walk” can be solved by: \
               “look right”, “look around right”, “look around right thrice”, “walk”. \
               Q: “turn right after run right thrice” \
               A: “turn right” can be solved by: “turn right”. “run right thrice” can be solved by: “run right”, “run \
               right thrice”. So, “turn right after run right thrice” can be solved by: “turn right”, “run right”, “run right \
               thrice”. \
               '

In [217]:
CD_Few_shot

'Q: “look right after look twice”                A: “look right after look twice” can be solved by: “look right”, “look twice”.                Q: “jump opposite right thrice and walk”                A: “jump opposite right thrice” can be solved by: “jump opposite right”, “jump opposite right thrice”.                “walk” can be solved by: “walk”. So, “jump opposite right thrice and walk” can be solved by: “jump                opposite right”, “jump opposite right thrice”, “walk”.                Q: “run left twice and run right”                A: “run left twice” can be solved by: “run left”, “run left twice”. “run right” can be solved by “run right”.                So, “run left twice and run right” can.be solved by: “run left”, “run left twice”, “run right”.                Q: “run opposite right”                A: “run opposite right” can be solved by “run opposite right”.                Q: “look opposite right thrice after walk”                A: “look opposite right thrice” can be 

In [218]:
CM_Few_shot = 'Q: “turn left” \
               A: “turn left” outputs “TURN LEFT”. \
               Q: “turn right” \
               A: “turn right” outputs “TURN RIGHT”. \
               Q: “jump left” \
               A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn left” \
               outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” is “TURN LEFT” + “JUMP”. \
               Q: “run right” \
               A: The output of “run right” concatenates: the output of “turn right”, the output of “run”. “turn right” \
               outputs “TURN RIGHT”. “run” outputs “RUN”. So concatenating the output of “turn right” and the \
               output of “run” leads to “TURN RIGHT” + “RUN”. So the output of “run right” is “TURN RIGHT” + \
               “RUN”. \
               Q: “look twice” \
               A: The output of “look twice” concatenates: the output of “look”, the output of “look”. “look” outputs \
               “LOOK”. So repeating the output of “look” two times leads to “LOOK” * 2. So the output of “look \
               twice” is “LOOK” * 2. \
               Q: “run and look twice” \
               A: The output of “run and look twice” concatenates: the output of “run”, the output of “look twice”. \
               “run” outputs “RUN”. “look twice” outputs “LOOK” * 2. So concatenating the output of “run” and the \
               output of “look twice” leads to “RUN” + “LOOK” * 2. So the output of “run and look twice” is “RUN” + \
               “LOOK” * 2. \
               Q: “jump right thrice” \
               A: The output of “jump right thrice” concatenates: the output of “jump right”, the output of “jump \
               right”, the output of “jump right”. “jump right” outputs “TURN RIGHT” + “JUMP”. So repeating the \
               output of “jump right” three times leads to (“TURN RIGHT” + “JUMP”) * 3. So the output of “jump \
               right thrice” is (“TURN RIGHT” + “JUMP”) * 3. \
               Q: “walk after run” \
               A: The output of “walk after run” concatenates: the output of “run”, the output of “walk”. “run” outputs \
               “RUN”. “walk” outputs “WALK”. So concatenating the output of “run” and the output of “walk” leads to \
               “RUN” + “WALK”. So the output of “walk after run” is “RUN” + “WALK”. \
               Q: “turn opposite left” \
               A: The output of “turn opposite left” concatenates: the output of “turn left”, the output of “turn left”. \
               “turn left” outputs “TURN LEFT”. So repeating the output of “turn left” twice leads to “TURN LEFT” * \
               2. So the output of “turn opposite left” is “TURN LEFT” * 2. \
               Q: “turn around left” \
               A: The output of “turn around left” concatenates: the output of “turn left”, the output of “turn left”, the \
               output of “turn left”, the output of “turn left”. “turn left” outputs “TURN LEFT”. So repeating the output \
               of “turn left” four times leads to “TURN LEFT” * 4. So the output of “turn around left” is “TURN LEFT” \
               * 4. \
               Q: “turn opposite right” \
               A: The output of “turn opposite right” concatenates: the output of “turn right”, the output of “turn \
               right”. “turn right” outputs “TURN RIGHT”. So repeating the output of “turn right” twice leads to \
               “TURN RIGHT” * 2. So the output of “turn opposite right” is “TURN RIGHT” * 2. \
               Q: “turn around right” \
               A: The output of “turn around right” concatenates: the output of “turn right”, the output of “turn right”, \
               the output of “turn right”, the output of “turn right”. “turn right” outputs “TURN RIGHT”. So repeating \
               the output of “turn right” four times leads to “TURN RIGHT” * 4. So the output of “turn around right” \
               is “TURN RIGHT” * 4. \
               Q: “walk opposite left” \
               A: The output of “walk opposite left” concatenates: the output of “turn opposite left”, the output of \
               “walk”. “turn opposite left” outputs “TURN LEFT” * 2. “walk” outputs “WALK”. So concatenating the \
               output of “turn opposite left” and the output of “walk” leads to “TURN LEFT” * 2 + “WALK”. So the \
               output of “walk opposite left” is “TURN LEFT” * 2 + “WALK”. \
               Q: “walk around left” \
               A: The output of “walk around left” concatenates: the output of “walk left”, the output of “walk left”, \
               the output of “walk left”, the output of “walk left”. “walk left” outputs “TURN LEFT” + “WALK”. So \
               repeating the output of “walk around left” four times leads to (“TURN LEFT” + “WALK”) * 4. So the \
               output of “walk around left” is (“TURN LEFT” + “WALK”) * 4. \
              '

In [219]:
CM_Few_shot

'Q: “turn left”                A: “turn left” outputs “TURN LEFT”.                Q: “turn right”                A: “turn right” outputs “TURN RIGHT”.                Q: “jump left”                A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn left”                outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” is “TURN LEFT” + “JUMP”.                Q: “run right”                A: The output of “run right” concatenates: the output of “turn right”, the output of “run”. “turn right”                outputs “TURN RIGHT”. “run” outputs “RUN”. So concatenating the output of “turn right” and the                output of “run” leads to “TURN RIGHT” + “RUN”. So the output of “run right” is “TURN RIGHT” +                “RUN”.                Q: “look twice”                A: The output of “look twice” concatenates: the out

换而言之，就是两个阶段设置这些示例的时候，能够让模型非常好的学习到相对完整的规则，并最终完成测试集的预测。

- 第一阶段提示工程

&emsp;&emsp;接下来就进入到命令拆解环节，我们可以通过如下代码快速实现命令拆解：

In [220]:
data

{'commands': 'turn opposite right thrice and turn opposite left',
 'actions': 'I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT'}

In [221]:
'Q：“%s” A:' % data['commands']

'Q：“turn opposite right thrice and turn opposite left” A:'

In [222]:
prompt_CD = CD_Few_shot + 'Q：“%s” A:' % data['commands']
prompt_CD

'Q: “look right after look twice”                A: “look right after look twice” can be solved by: “look right”, “look twice”.                Q: “jump opposite right thrice and walk”                A: “jump opposite right thrice” can be solved by: “jump opposite right”, “jump opposite right thrice”.                “walk” can be solved by: “walk”. So, “jump opposite right thrice and walk” can be solved by: “jump                opposite right”, “jump opposite right thrice”, “walk”.                Q: “run left twice and run right”                A: “run left twice” can be solved by: “run left”, “run left twice”. “run right” can be solved by “run right”.                So, “run left twice and run right” can.be solved by: “run left”, “run left twice”, “run right”.                Q: “run opposite right”                A: “run opposite right” can be solved by “run opposite right”.                Q: “look opposite right thrice after walk”                A: “look opposite right thrice” can be 

In [223]:
response_CD = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt_CD,
              temperature=0.5,
              max_tokens=1000
              )

In [224]:
response_CD["choices"][0]["text"].strip()

'“turn opposite right thrice” can be solved by: “turn opposite right”, “turn opposite right thrice”. “turn opposite left” can be solved by: “turn opposite left”. So, “turn opposite right thrice and turn opposite left” can be solved by: “turn opposite right”, “turn opposite right thrice”, “turn opposite left”.'

然后通过定义如下函数，完成拆解的子命令提取：

In [229]:
def extract_phrases(text):
    # 查找最后一个 "solved by:" 后面的所有内容
    last_solved_by = text.rsplit("solved by:", 1)[-1]
    
    # 使用正则表达式提取引号中的短语
    phrases = re.findall(r'“([^”]*)”', last_solved_by)
    
    return phrases

In [230]:
response_CD["choices"][0]["text"].strip()

'“turn opposite right thrice” can be solved by: “turn opposite right”, “turn opposite right thrice”. “turn opposite left” can be solved by: “turn opposite left”. So, “turn opposite right thrice and turn opposite left” can be solved by: “turn opposite right”, “turn opposite right thrice”, “turn opposite left”.'

In [231]:
CD_result = extract_phrases(response_CD["choices"][0]["text"].strip())
CD_result

['turn opposite right', 'turn opposite right thrice', 'turn opposite left']

- 第二阶段提示工程

In [256]:
# 字符串是不可变对象，通过等号即可完成深度复制
CM_Few_shot_temp = CM_Few_shot
CM_Few_shot_temp

'Q: “turn left”                A: “turn left” outputs “TURN LEFT”.                Q: “turn right”                A: “turn right” outputs “TURN RIGHT”.                Q: “jump left”                A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn left”                outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” is “TURN LEFT” + “JUMP”.                Q: “run right”                A: The output of “run right” concatenates: the output of “turn right”, the output of “run”. “turn right”                outputs “TURN RIGHT”. “run” outputs “RUN”. So concatenating the output of “turn right” and the                output of “run” leads to “TURN RIGHT” + “RUN”. So the output of “run right” is “TURN RIGHT” +                “RUN”.                Q: “look twice”                A: The output of “look twice” concatenates: the out

In [257]:
sub_qs = CD_result
sub_qs

['turn opposite right', 'turn opposite right thrice', 'turn opposite left']

In [258]:
from tqdm import tqdm

In [259]:
for qs in tqdm(sub_qs):
    print(qs)

100%|████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<?, ?it/s]

turn opposite right
turn opposite right thrice
turn opposite left


In [261]:
for qs in tqdm(sub_qs):
    CM_Few_shot_temp += 'Q:“%s” A：' % qs
    response_CM = openai.Completion.create(
                        model="text-davinci-003",
                        prompt=CM_Few_shot_temp,
                        temperature=0.5,
                        max_tokens=1000,
                        )
    CM_Few_shot_temp += response_CM["choices"][0]["text"].strip()

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:08<00:00,  2.97s/it]


In [247]:
CM_Few_shot_temp

'Q: “turn left”                A: “turn left” outputs “TURN LEFT”.                Q: “turn right”                A: “turn right” outputs “TURN RIGHT”.                Q: “jump left”                A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn left”                outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” is “TURN LEFT” + “JUMP”.                Q: “run right”                A: The output of “run right” concatenates: the output of “turn right”, the output of “run”. “turn right”                outputs “TURN RIGHT”. “run” outputs “RUN”. So concatenating the output of “turn right” and the                output of “run” leads to “TURN RIGHT” + “RUN”. So the output of “run right” is “TURN RIGHT” +                “RUN”.                Q: “look twice”                A: The output of “look twice” concatenates: the out

- 对原始问题提问：

In [262]:
data['commands']

'turn opposite right thrice and turn opposite left'

In [263]:
prompt_CM = CM_Few_shot_temp + 'Q：“%s” A:' % data['commands']
prompt_CM

'Q: “turn left”                A: “turn left” outputs “TURN LEFT”.                Q: “turn right”                A: “turn right” outputs “TURN RIGHT”.                Q: “jump left”                A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn left”                outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” is “TURN LEFT” + “JUMP”.                Q: “run right”                A: The output of “run right” concatenates: the output of “turn right”, the output of “run”. “turn right”                outputs “TURN RIGHT”. “run” outputs “RUN”. So concatenating the output of “turn right” and the                output of “run” leads to “TURN RIGHT” + “RUN”. So the output of “run right” is “TURN RIGHT” +                “RUN”.                Q: “look twice”                A: The output of “look twice” concatenates: the out

In [264]:
response_CM = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt_CM,
              temperature=0.5,
              max_tokens=1000,
              )

并提取最终预测结果：

In [266]:
CM_result = response_CM["choices"][0]["text"].strip()
CM_result

'The output of “turn opposite right thrice and turn opposite left” concatenates: the output of “turn opposite right thrice”, the output of “turn opposite left”. “turn opposite right thrice” outputs “TURN RIGHT” * 3. “turn opposite left” outputs “TURN LEFT” * 2. So concatenating the output of “turn opposite right thrice” and the output of “turn opposite left” leads to “TURN RIGHT” * 3 + “TURN LEFT” * 2. So the output of “turn opposite right thrice and turn opposite left” is (“TURN RIGHT” * 3) * 2 + “TURN LEFT” * 2.'

In [255]:
data

{'commands': 'turn opposite right thrice and turn opposite left',
 'actions': 'I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT'}

In [254]:
data['actions']

'I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT'

能够发现，模型能够顺利完成预测。接下来，通过如下函数进一步将CM_result输出结果转化为I_格式对象：

In [275]:
transform_expression(CM_result)

'I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_LEFT I_TURN_LEFT'

- 完整函数封装

&emsp;&emsp;接下来，我们把上述步骤进行完整流程的封装，这里需要提前准备两个阶段的Few-shot提示示例：

In [216]:
CD_Few_shot = 'Q: “look right after look twice” \
               A: “look right after look twice” can be solved by: “look right”, “look twice”. \
               Q: “jump opposite right thrice and walk” \
               A: “jump opposite right thrice” can be solved by: “jump opposite right”, “jump opposite right thrice”. \
               “walk” can be solved by: “walk”. So, “jump opposite right thrice and walk” can be solved by: “jump \
               opposite right”, “jump opposite right thrice”, “walk”. \
               Q: “run left twice and run right” \
               A: “run left twice” can be solved by: “run left”, “run left twice”. “run right” can be solved by “run right”. \
               So, “run left twice and run right” can.be solved by: “run left”, “run left twice”, “run right”. \
               Q: “run opposite right” \
               A: “run opposite right” can be solved by “run opposite right”. \
               Q: “look opposite right thrice after walk” \
               A: “look opposite right thrice” can be solved by: “look opposite right”, “look opposite right thrice”. \
               “walk” can be solved by “walk”. So, “look opposite right thrice after walk” can be solved by: “look \
               opposite right”, “look opposite right thrice”, “walk”. \
               Q: “jump around right” \
               A: “jump around right” can be solved by: “jump right”, “jump around right”. So, “jump around right” \
               can be solved by: “jump right”, “jump around right”. \
               Q: “look around right thrice and walk” \
               A: “look around right thrice” can be solved by: “look right”, “look around right”, “look around right \
               thrice”. “walk” can be solved by “walk”. So, “look around right thrice and walk” can be solved by: \
               “look right”, “look around right”, “look around right thrice”, “walk”. \
               Q: “turn right after run right thrice” \
               A: “turn right” can be solved by: “turn right”. “run right thrice” can be solved by: “run right”, “run \
               right thrice”. So, “turn right after run right thrice” can be solved by: “turn right”, “run right”, “run right \
               thrice”. \
               '

In [218]:
CM_Few_shot = 'Q: “turn left” \
               A: “turn left” outputs “TURN LEFT”. \
               Q: “turn right” \
               A: “turn right” outputs “TURN RIGHT”. \
               Q: “jump left” \
               A: The output of “jump left” concatenates: the output of “turn left”, the output of “jump”. “turn left” \
               outputs “TURN LEFT”. “jump” outputs “JUMP”. So concatenating the output of “turn left” and the output of “jump” leads to “TURN LEFT” + “JUMP”. So the output of “jump left” is “TURN LEFT” + “JUMP”. \
               Q: “run right” \
               A: The output of “run right” concatenates: the output of “turn right”, the output of “run”. “turn right” \
               outputs “TURN RIGHT”. “run” outputs “RUN”. So concatenating the output of “turn right” and the \
               output of “run” leads to “TURN RIGHT” + “RUN”. So the output of “run right” is “TURN RIGHT” + \
               “RUN”. \
               Q: “look twice” \
               A: The output of “look twice” concatenates: the output of “look”, the output of “look”. “look” outputs \
               “LOOK”. So repeating the output of “look” two times leads to “LOOK” * 2. So the output of “look \
               twice” is “LOOK” * 2. \
               Q: “run and look twice” \
               A: The output of “run and look twice” concatenates: the output of “run”, the output of “look twice”. \
               “run” outputs “RUN”. “look twice” outputs “LOOK” * 2. So concatenating the output of “run” and the \
               output of “look twice” leads to “RUN” + “LOOK” * 2. So the output of “run and look twice” is “RUN” + \
               “LOOK” * 2. \
               Q: “jump right thrice” \
               A: The output of “jump right thrice” concatenates: the output of “jump right”, the output of “jump \
               right”, the output of “jump right”. “jump right” outputs “TURN RIGHT” + “JUMP”. So repeating the \
               output of “jump right” three times leads to (“TURN RIGHT” + “JUMP”) * 3. So the output of “jump \
               right thrice” is (“TURN RIGHT” + “JUMP”) * 3. \
               Q: “walk after run” \
               A: The output of “walk after run” concatenates: the output of “run”, the output of “walk”. “run” outputs \
               “RUN”. “walk” outputs “WALK”. So concatenating the output of “run” and the output of “walk” leads to \
               “RUN” + “WALK”. So the output of “walk after run” is “RUN” + “WALK”. \
               Q: “turn opposite left” \
               A: The output of “turn opposite left” concatenates: the output of “turn left”, the output of “turn left”. \
               “turn left” outputs “TURN LEFT”. So repeating the output of “turn left” twice leads to “TURN LEFT” * \
               2. So the output of “turn opposite left” is “TURN LEFT” * 2. \
               Q: “turn around left” \
               A: The output of “turn around left” concatenates: the output of “turn left”, the output of “turn left”, the \
               output of “turn left”, the output of “turn left”. “turn left” outputs “TURN LEFT”. So repeating the output \
               of “turn left” four times leads to “TURN LEFT” * 4. So the output of “turn around left” is “TURN LEFT” \
               * 4. \
               Q: “turn opposite right” \
               A: The output of “turn opposite right” concatenates: the output of “turn right”, the output of “turn \
               right”. “turn right” outputs “TURN RIGHT”. So repeating the output of “turn right” twice leads to \
               “TURN RIGHT” * 2. So the output of “turn opposite right” is “TURN RIGHT” * 2. \
               Q: “turn around right” \
               A: The output of “turn around right” concatenates: the output of “turn right”, the output of “turn right”, \
               the output of “turn right”, the output of “turn right”. “turn right” outputs “TURN RIGHT”. So repeating \
               the output of “turn right” four times leads to “TURN RIGHT” * 4. So the output of “turn around right” \
               is “TURN RIGHT” * 4. \
               Q: “walk opposite left” \
               A: The output of “walk opposite left” concatenates: the output of “turn opposite left”, the output of \
               “walk”. “turn opposite left” outputs “TURN LEFT” * 2. “walk” outputs “WALK”. So concatenating the \
               output of “turn opposite left” and the output of “walk” leads to “TURN LEFT” * 2 + “WALK”. So the \
               output of “walk opposite left” is “TURN LEFT” * 2 + “WALK”. \
               Q: “walk around left” \
               A: The output of “walk around left” concatenates: the output of “walk left”, the output of “walk left”, \
               the output of “walk left”, the output of “walk left”. “walk left” outputs “TURN LEFT” + “WALK”. So \
               repeating the output of “walk around left” four times leads to (“TURN LEFT” + “WALK”) * 4. So the \
               output of “walk around left” is (“TURN LEFT” + “WALK”) * 4. \
              '

然后定义辅助函数，分别用于进行子问题的提取和最终问题结果的翻译：

In [294]:
def extract_phrases(text):
    # 查找最后一个 "solved by:" 后面的所有内容
    last_solved_by = text.rsplit("solved by:", 1)[-1]
    
    # 使用正则表达式提取引号中的短语
    phrases = re.findall(r'“([^”]*)”', last_solved_by)
    
    return phrases

In [269]:
import re

def transform_expression(s):
    # Regular expression pattern
    pattern = r'is “.*'

    # Find the match
    match = re.search(pattern, s) 
    
    s = match.group()[3: -1].replace('“', '"').replace('”', '"')
    # Step 1: Handle multiplications
    pattern = r'"([^"]+)" \* (\d+)'
    matches = re.findall(pattern, s)
    for match in matches:
        replacement = ' '.join([f'"{match[0]}"'] * int(match[1]))
        s = s.replace(f'"{match[0]}" * {match[1]}', replacement)

    # Step 2: Replace spaces within quotes with underscores
    pattern = r'"([^"]+)"'
    matches = re.findall(pattern, s)
    for match in matches:
        replacement = match.replace(' ', '_')
        s = s.replace(f'"{match}"', f'"{replacement}"')

    # Step 3: Add 'I_' prefix within quotes
    pattern = r'"([^"]+)"'
    matches = re.findall(pattern, s)
    for match in matches:
        replacement = 'I_' + match
        s = s.replace(f'"{match}"', f'"{replacement}"')

    # Step 4: Remove quotes
    s = s.replace('"', '')
    s = s.replace(' +', '')

    return s

然后是每条数据的预测过程：

In [293]:
def SCAN_predict(dataSet=scan_test, model="text-davinci-003", CD_Few_shot=CD_Few_shot, CM_Few_shot=CM_Few_shot):
    # 转化为dataframe
    data_frame = dataSet.to_pandas()
    # 最后一列标记为unkown
    data_frame['actions_predict'] = 'unkown'
    # 在字典中循环
    for i,data in enumerate(dataSet):
        # 阶段一：拆解命令
        prompt_CD = CD_Few_shot + 'Q：“%s” A:' % data['commands']
        response_CD = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt_CD,
              temperature=0.5,
              max_tokens=1000
              )
        # 拆解命令结果
        CD_result = extract_phrases(response_CD["choices"][0]["text"].strip())
        # 阶段二：短命令翻译
        CM_Few_shot_temp = CM_Few_shot
        sub_qs = CD_result
        for qs in sub_qs:
            CM_Few_shot_temp += 'Q:“%s” A：' % qs
            response_CM = openai.Completion.create(
                                model="text-davinci-003",
                                prompt=CM_Few_shot_temp,
                                temperature=0.5,
                                max_tokens=1000,
                                )
            CM_Few_shot_temp += response_CM["choices"][0]["text"].strip()
        # 对原始问题提问
        prompt_CM = CM_Few_shot_temp + 'Q：“%s” A:' % data['commands']
        response_CM = openai.Completion.create(
              model="text-davinci-003",
              prompt=prompt_CM,
              temperature=0.5,
              max_tokens=1000,
              )
        # 将结果保存在dataframe的对应位置
        data_frame['actions_predict'][i] = transform_expression(CM_result)
        
    return data_frame

In [286]:
data_frame = scan_test.to_pandas()
data_frame['actions_predict'] = 'unkown'

In [291]:
data_frame['actions_predict'][1]

'unkown'

In [287]:
data_frame

,commands,actions,actions_predict
0,turn opposite right thrice and turn opposite left,I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_...,unkown
1,run right twice after walk right twice,I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN...,unkown
2,look around right twice and turn left thrice,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN...,unkown
3,jump around left thrice and run right thrice,I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_L...,unkown
4,run thrice and walk opposite left,I_RUN I_RUN I_RUN I_TURN_LEFT I_TURN_LEFT I_WALK,unkown
...,...,...,...
4177,walk left and jump left twice,I_TURN_LEFT I_WALK I_TURN_LEFT I_JUMP I_TURN_L...,unkown
4178,jump around right twice after walk opposite ri...,I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT ...,unkown
4179,walk after jump opposite right twice,I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT ...,unkown
4180,look right thrice and jump left,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN...,unkown


最后，验证实际预测效果：

<span style="color:red">**下列代码谨慎运行，可能会造成大量的费用**</span>

In [297]:
data_frame = SCAN_predict(dataSet=scan_test)
data_frame

,commands,actions,actions_predict
0,turn opposite right thrice and turn opposite left,I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_...,I_TURN_RIGHT I_TURN_RIGHT I_TURN_RIGHT I_TURN_...
1,run right twice after walk right twice,I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN...,I_TURN_RIGHT I_WALK I_TURN_RIGHT I_WALK I_TURN...
2,look around right twice and turn left thrice,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN...,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN...
3,jump around left thrice and run right thrice,I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_L...,I_TURN_LEFT I_JUMP I_TURN_LEFT I_JUMP I_TURN_L...
4,run thrice and walk opposite left,I_RUN I_RUN I_RUN I_TURN_LEFT I_TURN_LEFT I_WALK,I_RUN I_RUN I_RUN I_TURN_LEFT I_TURN_LEFT I_WALK
...,...,...,...
4177,walk left and jump left twice,I_TURN_LEFT I_WALK I_TURN_LEFT I_JUMP I_TURN_L...,I_TURN_LEFT I_WALK I_TURN_LEFT I_JUMP I_TURN_L...
4178,jump around right twice after walk opposite ri...,I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT ...,I_TURN_RIGHT I_TURN_RIGHT I_WALK I_TURN_RIGHT ...
4179,walk after jump opposite right twice,I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT ...,I_TURN_RIGHT I_TURN_RIGHT I_JUMP I_TURN_RIGHT ...
4180,look right thrice and jump left,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN...,I_TURN_RIGHT I_LOOK I_TURN_RIGHT I_LOOK I_TURN...


In [298]:
data_frame['actions'] == data_frame['actions_predict']

0       True
1       True
2       True
3       True
4       True
        ... 
4177    True
4178    True
4179    True
4180    True
4181    True
Length: 4182, dtype: bool

In [307]:
(data_frame['actions'] == data_frame['actions_predict']).sum() / data_frame.shape[0]

1.0

In [306]:
data_frame.shape

(4182, 3)

能够发现，在最先进的completion模型text-davinci-003预测下，SCAN数据集的预测准确率能够达到100%。

### 4.Few-shot-LtM提示流程总结

&emsp;&emsp;接下来我们围绕上述Few-shot-LtM提示流程进行总结，并从理论层面，探讨该流程有效性的核心原因以及实践时的注意事项。

<center><img src="https://ml2022.oss-cn-hangzhou.aliyuncs.com/img/1688807852615.png" alt="1688807852615" style="zoom:33%;" />

&emsp;&emsp;首先需要说明的是，对于类似SCAN指令翻译这种复杂问题，很多时候Zero-shot都是无法解决的，此时必须要考虑Few-shot方法。而从一个更加抽象的角度来说，Few-shot中的提示示例，就更像是一个数据集中的训练数据，而Few-shot实际提示流程，就相当于是我们先让大模型学习训练数据来总结规律技巧，然后再进一步应用于新数据集上的预测。只不过和机器学习领域我们直接在完整数据集上随机划分训练集和测试集不同的是，这里的Few-shot往往不需要太多数据（受限于模型的上下文限制，也无法输入太多数据），但需要合理的挑选问答示例，并详细的展示由问到答的推理过程，才能够更好的给与模型启发。

&emsp;&emsp;当然，在实际执行过程中，往往是先确定提示流程、再编写Few-shot。提示工程流程是提示工程方法最高度的抽象，这方面创新难度较大，一般来说都是按照既有的提示框架进行完善和补充，就目前来看，LtM提示框架是最有效的提示框架。而在编写Few-shot的过程中，则是先从少部分数据中进行测试，刚开始编写和测试中间环节的问题和Few-shot会非常困难，往往都需要大模型工程师丰富的技术经验和一定的技术灵感。就本案例来说，在基于对英文语义充分理解的情况下，核心工作是以介词为界进行短命令处理，而第二个阶段的Few-shot则是先从不包含介词的短语入手进行指令翻译，然后尝试围绕带入介词的短语进行翻译，当模型能够理解部分介词+动词+副词的组合含义后，即可将其推广至更为一般的情况。

&emsp;&emsp;从这个过程也不难看出，实际上对于大语言模型来说，是具备非常强的“迁移学习”能力的，就好比一个成绩优秀的学生，其最大的潜力或许不在于现在某些知识掌握的非常好，而在于底层非常强大的学习能力和举一反三的能力，对于大语言模型亦是如此。对于类似SCAN数据集这种学习难度非常大的任务，模型仍然可以在只带入非常少的数据的情况下完成底层推理规则的学习，由此也确实能看出大语言模型恐怖的涌现能力。